In [11]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [12]:
df = pd.read_csv("DataResults/phases.csv")
df = df.iloc[: , 1:]
df

,PID,Starting Interruption,Starting Task,Condition,Control,d_age,d_gender,d_education,d_asd,d_colorblind,...,te_i_post_time,te_p_name,te_p_pre_correctness,te_p_pre_time,te_p_pre_resumption,te_p_pre_movestotal,te_p_post_correctness,te_p_post_time,te_p_post_resumption,te_p_post_movestotal
0,150657-543833,1,1,1,1,36,Female,University,False,False,...,1.802850,draw,7.25,13.300888,14.046581,NaN,6.0,18.245178,20.688902,NaN
1,189842-198434,1,1,1,1,25,Female,University,False,False,...,1.591400,draw,8.00,9.734822,9.734822,NaN,8.0,8.572062,8.572062,NaN
2,285256-429012,1,1,1,1,27,Male,High School,False,False,...,1.515050,draw,7.50,6.800664,7.561078,NaN,8.0,5.704932,5.704932,NaN
3,486637-941302,1,1,1,1,27,Male,University,False,False,...,1.291812,draw,8.00,8.066688,8.066688,NaN,7.5,10.059899,10.478063,NaN
4,522111-792046,1,1,1,1,24,Male,Masters,False,False,...,2.017275,draw,6.25,11.349828,11.381596,NaN,4.0,7.120666,8.752486,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,883881-672590,2,2,2,0,19,Male,High School,False,False,...,2.052713,hanoi,NaN,12.760413,1.020833,100.0,NaN,17.609862,1.109283,127.0
253,901440-351171,2,2,2,0,19,Female,High School,False,False,...,2.378625,hanoi,NaN,21.016286,1.175736,143.0,NaN,24.785474,1.313138,151.0
254,913584-192754,2,2,2,0,25,Female,Masters,False,False,...,5.299336,hanoi,NaN,27.225561,1.893952,115.0,NaN,31.303175,2.069631,121.0
255,967476-330431,2,2,2,0,25,Male,University,False,False,...,2.665913,hanoi,NaN,21.614099,1.764416,98.0,NaN,24.783737,1.709223,116.0


In [29]:
df = pd.read_csv("DataResults/phases.csv")
df_conditions = df.iloc[: , 1:][["Starting Interruption", "Starting Task"]]
df_conditions = df_conditions.drop_duplicates()

starting_interruption = []
starting_task = []
phases = []
metric_description = []
task_description = []
t_exp = []
p_exp = []
difference_exp = []
#t_control = []
#p_control = []
#difference_control = []
#t_condition = []
#p_condition = []
# entry = [] # condition, metric, task, t-exp, p-exp , difference-exp, t-control, p-control, difference-control

In [14]:
df_conditions

,Condition,Starting Interruption,Starting Task,Control
0,1,1,1,1
15,2,1,1,1
30,1,1,2,1
45,2,1,2,1
60,1,2,1,1
75,2,2,1,1
92,1,2,2,1
108,2,2,2,1
124,1,1,1,0
139,2,1,1,0


In [30]:
# helpers for interpreting t-test statistics

def is_improvement(change):
    if (change > 0):
        return "increased [+]"
    return "decreased [-]"

def is_significant(p):
    if (p <= 0.005):
        return "extremely significantly [**]"
    elif (p <= 0.05):
        return "significantly [*]"
    elif (p <= 0.03):
        return "slightly"
    return ""

def compute_improvement(col1, col2):
    col3 = col2 - col1
    avg = sum(col3)/len(col3)
    t, p = stats.ttest_ind(col1, col2)
    return t, p, avg

In [31]:
def compare_improvement(task, interruption, phase, metric_descript, df_data, pre_metric, post_metric):
    t_0, p_0, diff_0 = compute_improvement(
        df_data[(df_data['Starting Task'] == task) & (df_data['Starting Interruption'] == interruption)][pre_metric].astype(float), 
        df_data[(df_data['Starting Task'] == task) & (df_data['Starting Interruption'] == interruption)][post_metric].astype(float))
    
    print(t_0, p_0, diff_0)
    starting_interruption.append(interruption)
    starting_task.append(task)
    phases.append(phase)
    metric_description.append(metric_descript)
    t_exp.append(t_0)
    p_exp.append(p_0)
    difference_exp.append(diff_0)

In [32]:
for index, row in df_conditions.iterrows():
    if row['Starting Task'] == 1:
        compare_improvement(row['Starting Task'], row['Starting Interruption'], 'assessment', 'accuracy on primary task', df, 'a_p_pre_correctness', 'a_p_post_correctness')
        compare_improvement(row['Starting Task'], row['Starting Interruption'], 'training', 'accuracy on primary task', df, 'tr_p_pre_correctness', 'tr_p_post_correctness')
        compare_improvement(row['Starting Task'], row['Starting Interruption'], 'testing', 'accuracy on primary task', df, 'te_p_pre_correctness', 'te_p_post_correctness')
    else:
        compare_improvement(row['Starting Task'], row['Starting Interruption'], 'assessment', 'accuracy on primary task', df, 'a_p_pre_movestotal', 'a_p_post_movestotal')
        compare_improvement(row['Starting Task'], row['Starting Interruption'], 'training', 'accuracy on primary task', df, 'tr_p_pre_movestotal', 'tr_p_post_movestotal')
        compare_improvement(row['Starting Task'], row['Starting Interruption'], 'testing', 'accuracy on primary task', df, 'te_p_pre_movestotal', 'te_p_post_movestotal')
        
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'assessment', 'accuracy on interrupting task', df, 'a_i_pre_percentage', 'a_i_post_percentage')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'assessment', 'speed on primary task', df, 'a_p_pre_time', 'a_p_post_time')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'assessment', 'interruption lag / speed on interrupting task', df, 'a_i_pre_time', 'a_i_post_time')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'assessment', 'resumption lag', df, 'a_p_pre_resumption', 'a_p_post_resumption')
    
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'training', 'accuracy on interrupting task', df, 'tr_i_pre_percentage', 'tr_i_post_percentage')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'training', 'speed on primary task', df, 'tr_p_pre_time', 'tr_p_post_time')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'training', 'interruption lag / speed on interrupting task', df, 'tr_i_pre_time', 'tr_i_post_time')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'training', 'resumption lag', df, 'tr_p_pre_resumption', 'tr_p_post_resumption')
    
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'testing', 'accuracy on interrupting task', df, 'te_i_pre_percentage', 'te_i_post_percentage')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'testing', 'speed on primary task', df, 'te_p_pre_time', 'te_p_post_time')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'testing', 'interruption lag / speed on interrupting task', df, 'te_i_pre_time', 'te_i_post_time')
    compare_improvement(row['Starting Task'], row['Starting Interruption'], 'testing', 'resumption lag', df, 'te_p_pre_resumption', 'te_p_post_resumption')
    

-0.8504220751455553 0.3968127778353141 0.2
nan nan nan
0.12327608524215186 0.9020982060029401 -0.029166666666666667
0.7503975616499733 0.45450899181340887 -0.01875
1.8724667792506609 0.0636172390148845 -1.2342779054666664
2.5070064332450563 0.013536289141925057 -0.8272420294
2.206785920616559 0.029264515103511454 -1.5087771120000004
-0.6201337473880538 0.536365497481067 0.01875
1.2136088977193367 0.2273206880614326 -32.292710858166664
1.8702953749067477 0.06392063660849498 -0.4282387941333334
nan nan nan
-1.6202280837528802 0.10785272289725144 0.041666666666666664
0.12357353574209659 0.9018631892140345 -0.06872102639999995
2.7583229358619774 0.006735782201165144 -0.4705673343833333
-0.3287969669075282 0.7428919657710158 0.1683190250833335
8.33695408114825 9.473257303632765e-14 -76.42424242424242
nan nan nan
2.1145135858155792 0.03638072425493683 -11.151515151515152
0.8502145427380557 0.39676899269374666 -0.01893939393939394
4.834672503302577 3.701525583628662e-06 -6.071775947606058
4.8

c:\users\rramn\appdata\local\programs\python\python38\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\rramn\appdata\local\programs\python\python38\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\rramn\appdata\local\programs\python\python38\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\rramn\appdata\local\programs\python\python38\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\rramn\appdata\local\programs\python\python38\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\us

In [33]:
columnTitles = {
    "Starting Interruption": starting_interruption,
    "Starting Task": starting_task,
    "Phase": phases,
    "Metric": metric_description,
    "t": t_exp,
    "p": p_exp,
    "diff": difference_exp
}

dataframe = pd.DataFrame(columnTitles)
dataframe.to_csv('phases-task-interruption.csv')
print('DONE')

DONE
